In [1]:
import pandas as pd

In [19]:
# Read CSV into a DataFrame
csv_path = "SHR76_17.csv"
murder_df = pd.read_csv(csv_path)
murder_df.head()

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,...,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Alaska,Rural Alaska
1,197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
2,197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
3,197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
4,197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,Alabama,"Mobile, AL"


In [22]:
# Clean data up!
murder_weapons_series = murder_df['Weapon']

# Combine Gun categories
murder_weapons_series = murder_weapons_series.replace(to_replace=['Rifle', 'Handgun', 'Firearm', 'Shotgun', 'Other Gun'], value='Gun')
murder_df = murder_df.assign(Weapon=murder_weapons_series)

# Combine Circumstances
murder_circumstance_series = murder_df['Circumstance']
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Other arguments', 'Argument over money or property'], value='Arguments')
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Other negligent handling of gun', 'Children playing with gun', 'Gun-cleaning death - other than self', 'Victim shot in hunting accident'], value='Gun Negligence')
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Rape', 'Other sex offense'], value='Sex Offense')
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Circumstances undetermined', 'Other', 'Other - not specified', 'All other manslaughter by negligence', 'All suspected felony type'], value='Other')
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Robbery', 'Burglary', 'Motor vehicle theft', 'Larceny'], value='Theft')
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Brawl due to influence of alcohol', 'Brawl due to influence of narcotics', 'Narcotic drug laws'], value='Drug & Alcohol Related')
murder_circumstance_series = murder_circumstance_series.replace(to_replace=['Gangland killings', 'Juvenile gang killings'], value='Gang Related')
murder_df = murder_df.assign(Circumstance=murder_circumstance_series)

murder_relationship_series = murder_df['Relationship']
murder_relationship_series = murder_relationship_series.replace(to_replace=['Husband', 'Wife', 'Common-law husband', 'Common-law wife'], value='Spouse')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Brother', 'Sister'], value='Sibling')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Mother', 'Father'], value='Parent')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Ex-husband', 'Ex-wife'], value='Ex')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Stepmother', 'Stepfather'], value='Stepparent')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Stepdaughter', 'Stepson'], value='Stepchild')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Boyfriend', 'Girlfriend'], value='Partner')
murder_relationship_series = murder_relationship_series.replace(to_replace=['Relationship not determined', 'Other - known to victim'], value='Other')
murder_df = murder_df.assign(Relationship=murder_relationship_series)

murder_df.head()

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,...,Unknown or not reported,Knife or cutting instrument,Spouse,Arguments,NaN,0,0,30180.0,Alaska,Rural Alaska
1,197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Gun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
2,197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Gun,Spouse,Other,NaN,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
3,197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Knife or cutting instrument,Sibling,Arguments,NaN,0,0,30180.0,Alabama,"Birmingham-Hoover, AL"
4,197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,...,Unknown or not reported,Strangulation - hanging,Acquaintance,Other,NaN,0,0,30180.0,Alabama,"Mobile, AL"
